In [1]:
import json
import os
import pandas as pd
import glob
import pandasgui as pdg

print(pdg.__version__)
diff_svg_threshold = 1e-4

0.2.13


In [5]:
dataset = ["OpenAPS_NS", "OPENonOH"]
outdir = "/home/reinhold/Daten/OPEN/"
outfilename = f"duplicates_{dataset[0]}_{dataset[1]}.csv"

indir = [f"{outdir}/{d}_Data/" for d in dataset]
infilename = [f"{d}_per_day.csv" for d in dataset]

df = [pd.read_csv(os.path.join(indir[i], infilename[i]), header=0, parse_dates=[1], index_col=0) for i in range(2)]
df[0]["date"] = pd.to_datetime(df[0]["date"],format="%Y-%m-%d")
out = df[0].merge(df[1], left_on="date", right_on="date", how="inner", suffixes=("_ds1", "_ds2"))

print(df[0].columns)
for col in df[0].columns:
    if not "sgv" in col: continue
    out["diff_" + col] = 2*(out[col + "_ds1"] - out[col + "_ds2"])/(out[col + "_ds1"] + out[col + "_ds2"])

df2 = out[(out.diff_sgv_mean*out.diff_sgv_mean < diff_svg_threshold) & (out.diff_sgv_std*out.diff_sgv_std < diff_svg_threshold) & 
    (out.diff_sgv_min*out.diff_sgv_min < diff_svg_threshold) & (out.diff_sgv_max*out.diff_sgv_max < diff_svg_threshold) & 
    (out.diff_sgv_count*out.diff_sgv_count < diff_svg_threshold)]
df3 = df2[["user_id_ds1", "user_id_ds2", "date", "diff_sgv_mean", "diff_sgv_std", "diff_sgv_min", "diff_sgv_max", "diff_sgv_count", 
    "filename_ds1", "filename_ds2"]].sort_values(by=["user_id_ds1", "user_id_ds2", "date"])
#df3 = df3[[]]

# group by statement has to return exactly one line, otherwise the selection criteria on diff_sgv_*-variables need to be tightened.

#gui = pdg.show(df3)

df3.to_csv(os.path.join(outdir, outfilename))
print(os.path.join(outdir, outfilename) + " created")


Index(['date', 'sgv_mean', 'sgv_std', 'sgv_min', 'sgv_max', 'sgv_count',
       'filename', 'user_id'],
      dtype='object')
/home/reinhold/Daten/OPEN/duplicates_OpenAPS_NS_OPENonOH.csv created


In [10]:
df_duplicates_only = df3.groupby(["diff_sgv_mean", "diff_sgv_std", "diff_sgv_min", "diff_sgv_max", "diff_sgv_count"]).agg(["count"])
if len(df_duplicates_only) > 1: 
    columns = []
    for col in df_duplicates_only.columns:
        columns.append(f"{col[0]}_{col[1]}")
    df_duplicates_only.columns = columns
    df_duplicates_only.fillna(value=0, inplace=True)  #
    df_duplicates_only.reset_index(inplace=True)

    print(df_duplicates_only)
    print("all diff*-entries in df_duplicates_only need to be very close to 0.")
    print(type(df_duplicates_only[["diff_sgv_mean", "diff_sgv_std"]]))
    print(type(df_duplicates_only["diff_sgv_mean"]))
    if df_duplicates_only.any() > diff_svg_threshold: raise ValueError()

    diff_sgv_mean  diff_sgv_std  diff_sgv_min  diff_sgv_max  diff_sgv_count  \
0             0.0 -1.590998e-15           0.0           0.0             0.0   
1             0.0 -6.462215e-16           0.0           0.0             0.0   
2             0.0 -6.194471e-16           0.0           0.0             0.0   
3             0.0 -4.162595e-16           0.0           0.0             0.0   
4             0.0 -2.927725e-16           0.0           0.0             0.0   
5             0.0 -2.550783e-16           0.0           0.0             0.0   
6             0.0 -1.691355e-16           0.0           0.0             0.0   
7             0.0  0.000000e+00           0.0           0.0             0.0   
8             0.0  1.170890e-16           0.0           0.0             0.0   
9             0.0  1.173453e-16           0.0           0.0             0.0   
10            0.0  1.847028e-16           0.0           0.0             0.0   
11            0.0  3.136344e-16           0.0       

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [4]:
# get list of duplicates and the count of days
df3_mod = df3[['user_id_ds1', 'user_id_ds2','date']]
df4 = df3_mod.groupby(['user_id_ds1', 'user_id_ds2']).agg(["count"])

columns = []
for col in df4.columns:
    columns.append(f"{col[0]}_{col[1]}")
df4.columns = columns
df4.fillna(value=0, inplace=True)  #
df4.reset_index(inplace=True)
#print(df4[['user_id_ds1', 'user_id_ds2','date_count']])
print(df4)


    user_id_ds1  user_id_ds2  date_count
0      14092221     20104403         590
1      15634563     34208593         134
2      24448124     50641121         795
3      27526291     77770642          38
4      54280576      8814820          15
5      63725802      1104400         256
6      67167655     26745937         189
7      68026248     45407256         223
8      70454270     72492017         203
9      80625186     77667622         330
10     80796147     65054673         281
11     86323903       749582        1300
12     89032650     99868978         346
13     90398368     54943175          55
14     93606058      9922617         359
15     94200862     12826270          51
16     98340749     21316466         583
